In [1]:
import numpy as np
import torch
import cv2
import tifffile as tff
import matplotlib.pyplot as plt
import os
import glob
import matplotlib as mpl
import nibabel as nib
mpl.rcParams['figure.dpi']= 100

In [2]:
def test_normalization(image, label, exp_name, CLAHE = False, clipLimit = 3.5, tileGridSize = (8,8), gaussian_blur = False, ksize = (3,3), sigma = 0.5, verbose = False, save_images = False):
    if CLAHE:
        clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
        image = clahe.apply(image)
    if gaussian_blur:
        image = cv2.GaussianBlur(src = image, ksize = ksize, sigmaX = sigma)

    image = (image-127.5)/127.5 #center data
#         self.label = self.label/255 #binarize
    label = label != 0
#         self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
    
    if verbose:
        
        X1 = image[0:160,0:160]
        X2 = image[160:320,160:320]
        Y1 = label[0:160,0:160]
        Y2 = label[160:320,160:320]
        plt.imshow(image, cmap = "gray")
        plt.figure()
        plt.hist(image.reshape(-1), bins = 10, range = (-1,1))
        plt.figure()
        plt.imshow(X1, cmap = "gray")
        plt.figure()
        plt.imshow(Y1*255, cmap = "gray")
        plt.figure()
        plt.imshow(X2, cmap = "gray")
        plt.figure()
        plt.imshow(Y2*255, cmap = "gray")
        
        
        
    if save_images:
        X1 = image[0:160,0:160]
        X2 = image[160:320,160:320]
        Y1 = label[0:160,0:160]
        Y2 = label[160:320,160:320]
        
        cv2.imwrite("normalization_results/" + str(exp_name) + "/CLAHE_" + str(CLAHE) + "_clipLimit_" + str(clipLimit) + "ksize_" + str(ksize) + "tileGridSize_" + str(tileGridSize) + "gaussian_blur_" + str(gaussian_blur) + "sigma_" + str(sigma) + "_X1.png" ,X1)
        cv2.imwrite("normalization_results/" + str(exp_name) + "/CLAHE_" + str(CLAHE) + "_clipLimit_" + str(clipLimit) + "ksize_" + str(ksize) + "tileGridSize_" + str(tileGridSize) + "gaussian_blur_" + str(gaussian_blur) + "sigma_" + str(sigma) + "_X2.png" ,X2)
        cv2.imwrite("normalization_results/" + str(exp_name) + "/CLAHE_" + str(CLAHE) + "_clipLimit_" + str(clipLimit) + "ksize_" + str(ksize) + "tileGridSize_" + str(tileGridSize) + "gaussian_blur_" + str(gaussian_blur) + "sigma_" + str(sigma) + "_Y1.png" ,Y1.astype(np.uint8))
        cv2.imwrite("normalization_results/" + str(exp_name) + "/CLAHE_" + str(CLAHE) + "_clipLimit_" + str(clipLimit) + "ksize_" + str(ksize) + "tileGridSize_" + str(tileGridSize) + "gaussian_blur_" + str(gaussian_blur) + "sigma_" + str(sigma) + "_Y2.png" ,Y2.astype(np.uint8))
        
    return(image, label)
        

In [2]:
class SNEMI3DDataset_Train(torch.utils.data.Dataset):
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = tff.imread("/home/xrioen/scratch/SNEMI3D_raw/train-input.tif")[0:72,:,:]
        self.label = tff.imread("/home/xrioen/scratch/SNEMI3D_raw/train-labels.tif")[0:72,:,:]
        self.kernel = np.ones((3,3),np.uint8)

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(72):
            if CLAHE:
                image1 = self.dataset[i,:,:]
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            
            erode0 = cv2.erode(self.label[i,:,:],self.kernel,iterations = 1)
            dilate0 = cv2.dilate(self.label[i,:,:],self.kernel,iterations = 1)
            self.label[i,:,:][dilate0!=erode0] = 0
            
        
        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        self.label = self.label != 0
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
#         self.Y[self.Y >= 0.5] = 1  #if reshape is used, make sure labels are binary for torchmetrics
#         self.Y[self.Y < 0.5] = 0 
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        
        
class SNEMI3DDataset_Test(torch.utils.data.Dataset):
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = tff.imread("/home/xrioen/scratch/SNEMI3D_raw/train-input.tif")[-18:,:,:]
        self.label = tff.imread("/home/xrioen/scratch/SNEMI3D_raw/train-labels.tif")[-18:,:,:]
        self.kernel = np.ones((3,3),np.uint8)

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(18):
            if CLAHE:
                image1 = self.dataset[i,:,:]
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
                
            
            erode0 = cv2.erode(self.label[i,:,:],self.kernel,iterations = 1)
            dilate0 = cv2.dilate(self.label[i,:,:],self.kernel,iterations = 1)
            self.label[i,:,:][dilate0!=erode0] = 0
            
        
        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        self.label = self.label != 0
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
#         self.Y[self.Y >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
#         self.Y[self.Y < 0.5] = 0 
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)

In [3]:
class UrocellDataset_mito_block(torch.utils.data.Dataset):
     #fib1-0-0-0.nii.gz  fib1-1-0-3.nii.gz  fib1-3-2-1.nii.gz  fib1-3-3-0.nii.gz  fib1-4-3-0.nii.gz
    def __init__(self, block_id = ["fib1-0-0-0.nii.gz"], stride = (9,80,80), CLAHE = True):
        self.data_ids = block_id
        self.X = torch.empty((0,1,18,160,160))
        self.Y = torch.empty((0,1,18,160,160))
        
        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))
            
        for d in self.data_ids:
            self.dataset0 = nib.load("/home/xrioen/scratch/UroCell-master/data/" + d).get_fdata().astype(np.uint8)
            self.label0 = nib.load("/home/xrioen/scratch/UroCell-master/mito/binary/" + d).get_fdata().astype(np.uint8) #already binary
            
            self.dataset = np.zeros((256,400,400))
            self.label = np.zeros((256,400,400))

            for i in range(256):
                if CLAHE:
                    image1 = self.dataset0[i,:,:]
                    image1 = self.clahe.apply(image1)
                    self.dataset0[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
                
                self.dataset[i,:,:] = cv2.resize(self.dataset0[i,:,:], (400,400), interpolation = cv2.INTER_CUBIC) #enlarging = cubic,linear, shrinking = area
                self.label[i,:,:] = cv2.resize(self.label0[i,:,:], (400,400), interpolation = cv2.INTER_CUBIC)

            self.dataset = (self.dataset-127.5)/127.5 #center data
            self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
            self.label = torch.Tensor(self.label).unsqueeze(0)

            self.fov = (18,160,160)
            self.stride = stride

            #tiling
            self.X0 = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
            self.X0 = self.X0.reshape(-1,18,160,160).unsqueeze(1)
            print("Images parsed into tensor of size:", self.X0.size())
            self.Y0 = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
            self.Y0 = self.Y0.reshape(-1,18,160,160).unsqueeze(1)
            self.Y0[self.Y0 >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
            self.Y0[self.Y0 < 0.5] = 0 
            self.Y0 = self.Y0.int()
            print("Labels parsed into tensor of size:", self.Y0.size())
            
            self.X = torch.cat((self.X,self.X0),dim = 0)
            self.Y = torch.cat((self.Y,self.Y0),dim = 0)


    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)

In [2]:
class RatLiverDataset_mito_Train18(torch.utils.data.Dataset):
    #assume seg masks is binary [0, 255]
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((18,1000,1000))
        self.label = np.zeros((18,1000,1000))

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(18):
            
            ind = i + 19
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/EM20-324-200_series_processed/' + 
                                                    str(ind+1).zfill(3) + "____z" + str(ind) + '.0.tif')[0:3500,0:3500], (1000,1000), interpolation = cv2.INTER_AREA)
            self.label[i,:,:] = cv2.resize(cv2.imread('/home/xrioen/scratch/rat_mito_gt_exported/Slice 54 (1).vsseg_export_s' + 
                                                    str(ind).zfill(3)+ '.png',0), (1000,1000), interpolation = cv2.INTER_AREA)
            
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1

        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        if np.max(self.label) == 255:
            print("Scaling masks to [0,1] from [0,255]")
            self.label = self.label/255 #constrict range from 0 to 1
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        

In [3]:
class RatLiverDataset_mito_Test18(torch.utils.data.Dataset):
    #goes from slices 37-54

    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((18,1000,1000))
        self.label = np.zeros((18,1000,1000))

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(18):
            
            ind = i + 37
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/EM20-324-200_series_processed/' + 
                                                    str(ind+1).zfill(3) + "____z" + str(ind) + '.0.tif')[0:3500,0:3500], (1000,1000), interpolation = cv2.INTER_AREA)
            self.label[i,:,:] = cv2.resize(cv2.imread('/home/xrioen/scratch/rat_mito_gt_exported/Slice 54 (1).vsseg_export_s' + 
                                                    str(ind).zfill(3)+ '.png',0), (1000,1000), interpolation = cv2.INTER_AREA)
            
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1

        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        if np.max(self.label) == 255:
            print("Scaling masks to [0,1] from [0,255]")
            self.label = self.label/255 #constrict range from 0 to 1
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        

In [2]:
class RatLiverDataset_ER_18_35(torch.utils.data.Dataset):

    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((18,1000,1000))
        self.label = np.zeros((18,1000,1000))

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(18):
            
            ind = i + 18
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/EM20-324-200_series_processed/' + 
                                                    str(ind+1).zfill(3) + "____z" + str(ind) + '.0.tif')[0:3500,0:3500], (1000,1000), interpolation = cv2.INTER_AREA)
            self.label[i,:,:] = cv2.resize(cv2.imread('/home/xrioen/scratch/rat_liver_ER_fez_18_35/Slice 3-4 + 18-35 (Updated).vsseg_export_s' + 
                                                    str(ind).zfill(3)+ '.png',0), (1000,1000), interpolation = cv2.INTER_AREA)
            
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1

        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        if np.max(self.label) == 255:
            print("Scaling masks to [0,1] from [0,255]")
            self.label = self.label/255 #constrict range from 0 to 1
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        

In [2]:
class RatLiverDataset_ER_18_35_train(torch.utils.data.Dataset):
    #the top 560/1000 is training set
    #the bottom 440/1000 is testing
    #! make sure stride is 40

    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((18,1000,560))
        self.label = np.zeros((18,1000,560))

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(18):
            
            ind = i + 18
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/EM20-324-200_series_processed/' + 
                                                    str(ind+1).zfill(3) + "____z" + str(ind) + '.0.tif')[0:3500,0:3500], (1000,1000), interpolation = cv2.INTER_AREA)[:,0:560]
            self.label[i,:,:] = cv2.resize(cv2.imread('/home/xrioen/scratch/rat_liver_ER_fez_18_35/Slice 3-4 + 18-35 (Updated).vsseg_export_s' + 
                                                    str(ind).zfill(3)+ '.png',0), (1000,1000), interpolation = cv2.INTER_AREA)[:,0:560]
            
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1

        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        if np.max(self.label) == 255:
            print("Scaling masks to [0,1] from [0,255]")
            self.label = self.label/255 #constrict range from 0 to 1
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        

In [3]:
class RatLiverDataset_ER_18_35_test(torch.utils.data.Dataset):
    #the top 560/1000 is training set
    #the bottom 440/1000 is testing
    #! make sure stride is 40

    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((18,1000,440))
        self.label = np.zeros((18,1000,440))

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(18):
            
            ind = i + 18
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/EM20-324-200_series_processed/' + 
                                                    str(ind+1).zfill(3) + "____z" + str(ind) + '.0.tif')[0:3500,0:3500], (1000,1000), interpolation = cv2.INTER_AREA)[:,560:]
            self.label[i,:,:] = cv2.resize(cv2.imread('/home/xrioen/scratch/rat_liver_ER_fez_18_35/Slice 3-4 + 18-35 (Updated).vsseg_export_s' + 
                                                    str(ind).zfill(3)+ '.png',0), (1000,1000), interpolation = cv2.INTER_AREA)[:,560:]
            
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1

        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        if np.max(self.label) == 255:
            print("Scaling masks to [0,1] from [0,255]")
            self.label = self.label/255 #constrict range from 0 to 1
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        

In [4]:
class RatLiverDataset_all(torch.utils.data.Dataset):
    #assume seg masks is binary [0, 255]
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((219,2000,2000))
        self.label = np.zeros((219,2000,2000))

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))
        
        for i in range(219):
            image1 =  cv2.resize(tff.imread('/home/xrioen/scratch/EM20-324-200_series_processed/' + 
                                                    str(i+1).zfill(3) + "____z" + str(i) + '.0.tif'), (2000,2000))
#             self.label[i,:,:] = cv2.resize(cv2.imread('/home/xrioen/scratch/rat_mito_gt_exported/Slice 54 (1).vsseg_export_s' + 
#                                                     str(ind).zfill(3)+ '.png',0), (1000,1000), interpolation = cv2.INTER_AREA)
            
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1

        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
#         if np.max(self.label) == 255:
#             print("Scaling masks to [0,1] from [0,255]")
#             self.label = self.label/255 #constrict range from 0 to 1
#         self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
#         self.label[self.label < 0.5] = 0 
#         self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
#         self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
#         self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
#         self.Y = self.Y.int()
#         print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.X.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
#         plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
#         plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
#         plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
#         print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        

In [2]:
class HarvardLiverDataset_mito_Train180(torch.utils.data.Dataset):
    #assume seg masks is binary [0, 255]
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((180,2000,3000))
        self.label = np.zeros((180,2000,3000))
        self.kernel = np.ones((3,3),np.uint8)

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(180):
            ind1 = i*5 + 1000
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/6461_Raw/Gunes_WT1_8x8x8nm_3MHz.' + str(ind1).zfill(4) + '.tif'), (3000,2000), interpolation = cv2.INTER_AREA)
            self.label[i,:,:] = cv2.resize(tff.imread('/home/xrioen/scratch/6461_Mito_filled/' + str(ind1).zfill(4) + '.tiff'), (3000,2000), interpolation = cv2.INTER_AREA)
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1
            
        
        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        self.label = self.label/255 
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)

class HarvardLiverDataset_mito_Test54(torch.utils.data.Dataset):
    #assume seg masks is binary [0, 255]
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((180,2000,3000))
        self.label = np.zeros((180,2000,3000))
        self.kernel = np.ones((3,3),np.uint8)

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(54):
            ind1 = i*5 + 2000
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/6461_Raw/Gunes_WT1_8x8x8nm_3MHz.' + str(ind1).zfill(4) + '.tif'), (3000,2000))
            self.label[i,:,:] = cv2.resize(tff.imread('/home/xrioen/scratch/6461_Mito_filled/' + str(ind1).zfill(4) + '.tiff'), (3000,2000))
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1
        
        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        self.label = self.label/255 
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        

In [3]:
class HarvardLiverDataset_ER_Train180(torch.utils.data.Dataset):
    #assume seg masks is binary [0, 255]
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((180,2000,3000))
        self.label = np.zeros((180,2000,3000))
        self.kernel = np.ones((3,3),np.uint8)

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(180):
            ind1 = i*5 + 1000
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/6461_Raw/Gunes_WT1_8x8x8nm_3MHz.' + str(ind1).zfill(4) + '.tif'), (3000,2000))
            self.label[i,:,:] = cv2.resize(tff.imread('/home/xrioen/scratch/6461_ER/er_' + str(ind1).zfill(6) + '.tiff'), (3000,2000))
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1
            
        
        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        self.label = self.label/255 
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)

class HarvardLiverDataset_ER_Test54(torch.utils.data.Dataset):
    #assume seg masks is binary [0, 255]
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((180,2000,3000))
        self.label = np.zeros((180,2000,3000))
        self.kernel = np.ones((3,3),np.uint8)

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(54):
            ind1 = i*5 + 2000
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/6461_Raw/Gunes_WT1_8x8x8nm_3MHz.' + str(ind1).zfill(4) + '.tif'), (3000,2000))
            self.label[i,:,:] = cv2.resize(tff.imread('/home/xrioen/scratch/6461_ER/er_' + str(ind1).zfill(6) + '.tiff'), (3000,2000))
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1
        
        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        self.label = self.label/255 
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        

In [4]:
class HarvardLiverDataset_LD_Train180(torch.utils.data.Dataset):
    #assume seg masks is binary [0, 255]
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((180,2000,3000))
        self.label = np.zeros((180,2000,3000))
        self.kernel = np.ones((3,3),np.uint8)

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(180):
            ind1 = i*5 + 1000
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/6461_Raw/Gunes_WT1_8x8x8nm_3MHz.' + str(ind1).zfill(4) + '.tif'), (3000,2000))
            self.label[i,:,:] = cv2.resize(tff.imread('/home/xrioen/scratch/6461_LD/lipid_' + str(ind1).zfill(6) + '.tiff'), (3000,2000))
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1
            
        
        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        self.label = self.label/255 
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)

class HarvardLiverDataset_LD_Test54(torch.utils.data.Dataset):
    #assume seg masks is binary [0, 255]
    def __init__(self, stride = (9,80,80), CLAHE = True):
        'Initialization'
        self.dataset = np.zeros((180,2000,3000))
        self.label = np.zeros((180,2000,3000))
        self.kernel = np.ones((3,3),np.uint8)

        if CLAHE:
            self.clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8,8))

        for i in range(54):
            ind1 = i*5 + 2000
            image1 = cv2.resize(tff.imread('/home/xrioen/scratch/6461_Raw/Gunes_WT1_8x8x8nm_3MHz.' + str(ind1).zfill(4) + '.tif'), (3000,2000))
            self.label[i,:,:] =  cv2.resize(tff.imread('/home/xrioen/scratch/6461_LD/lipid_' + str(ind1).zfill(6) + '.tiff'), (3000,2000))
            if CLAHE:
                image1 = self.clahe.apply(image1)
                self.dataset[i,:,:] = cv2.GaussianBlur(src = image1, ksize = (3,3), sigmaX = 0.5)
            else:
                self.dataset[i,:,:] = image1
        
        self.dataset = (self.dataset-127.5)/127.5 #center data
        self.dataset = torch.Tensor(self.dataset).unsqueeze(0)
        
        self.label = self.label/255 
        self.label[self.label >= 0.5] = 1  #if resize is used, make sure labels are binary for torchmetrics
        self.label[self.label < 0.5] = 0 
        self.label = torch.Tensor(self.label.astype(np.int8)).unsqueeze(0)
        
        self.fov = (18,160,160)
        self.stride = stride
        
        #tiling
        self.X = self.dataset.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.X = self.X.reshape(-1,18,160,160).unsqueeze(1)
        print("Images parsed into tensor of size:", self.X.size())
        self.Y = self.label.unfold(1,self.fov[0],self.stride[0]).unfold(2,self.fov[1],self.stride[1]).unfold(3,self.fov[2],self.stride[2])
        self.Y = self.Y.reshape(-1,18,160,160).unsqueeze(1)
        self.Y = self.Y.int()
        print("Labels parsed into tensor of size:", self.Y.size())
        

    def __len__(self):
        'Denotes the total number of samples'
        return (self.Y.shape[0])

    def __getitem__(self, index):
        #index is gapped every 18
        'Generates one sample of data'
        return self.X[index], self.Y[index]
    
    def sanity_check(self):
        plt.figure()
        plt.hist(np.array(self.X)[0:20,0,0,:,:].reshape(-1), bins = 20)
        plt.figure()
        plt.imshow(self.X[0,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[0,0,0,:,:]*255, cmap = "gray")
        plt.figure()
        plt.imshow(self.X[10,0,0,:,:], cmap = "gray")
        plt.figure()
        plt.imshow(self.Y[10,0,0,:,:]*255, cmap = "gray")
        print("unique Y:", np.unique(self.Y, return_counts = True)) #sanity check to see if there are other labels, and contain only 0 and 1, with 1 as the foregronud
        print("X max:", torch.max(self.X[:,:,:,:,:]))
        print("X min:", torch.min(self.X[:,:,:,:,:]))
        print("X shape:", self.X.shape)
        